# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [94]:
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!
# First, put in your user name below
#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!#!

MYTABLE='kodurus' + '_data'

import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd


host = 'slucor2020.cgdcoitnku0k.us-east-1.rds.amazonaws.com'

port = '3306'

user = 'slucor2020'

password = 'SLUcor2020'

database = 'hds5210'

# Then, create a connection to the same database, using the 
# same user name and password in the `mysql-examples.ipynb` 
# file.  Use conn as the database connection variable.

conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)


In [95]:
assert(MYTABLE != 'paulboal_data')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [96]:
import pandas as pd
url = 'http://dhcs-chhsagency.opendata.arcgis.com/datasets/8e4f3a0c75b9424d888d11c1f949cc32_0.csv'
data = pd.read_csv(url)
display(data)
### SOLUTION GOES HERE



,Provider_Name,NPI,CCN,Medicaid_EP_Hospital_Type,Street_Address,City,County,State,Zip_Code,Payment_Year_Number,Program_Type,Total_payments,Last_Program_Year,Last_Payment_Year,Last_Payment_Criteria,Most_Recent_Disbursement_Amount,LONGITUDE,LATITUDE,FID
0,Sutter Bay Hospitals,1659439834,50008,Acute Care Hospitals,CASTRO & DUBOCE STS,SAN FRANCISCO,San Francisco,CA,94114,3,Medicare/Medicaid,638474,2015,2017,MU,70942,-122.435802,37.769062,1
1,PRIME HEALTHCARE SERVICES - GARDEN GROVE LLC,1659538858,50230,Acute Care Hospitals,12601 GARDEN GROVE BLVD,GARDEN GROVE,Orange,CA,92843,4,Medicare/Medicaid,3947489,2014,2015,MU,394749,-117.913856,33.774499,2
2,ST MARY MEDICAL CENTER,1669456299,50300,Acute Care Hospitals,18300 US HIGHWAY 18,APPLE VALLEY,San Bernardino,CA,92307,3,Medicare/Medicaid,3062645,2014,2015,MU,340294,-117.262672,34.539918,3
3,MADERA COMMUNITY HOSPITAL,1669673646,50568,Acute Care Hospitals,1250 E ALMOND AVE,MADERA,Madera,CA,93637,4,Medicare/Medicaid,2057365,2015,2016,MU,205737,-120.045618,36.945447,4
4,Temecula Valley Hospital Inc,1679816201,50775,Acute Care Hospitals,31700 TEMECULA PKWY,TEMECULA,Riverside,CA,92592,1,Medicare/Medicaid,474790,2016,2017,AIU,474790,-117.117197,33.470664,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,KAISER FOUNDATION HOSPITALS,1326119967,50071,Acute Care Hospitals,700 LAWRENCE EXPY,SANTA CLARA,Santa Clara,CA,95051,1,Medicare/Medicaid,339080,2016,2017,MU,339080,-121.995774,37.334574,324
324,COUNTY OF LOS ANGELES AUDITOR CONTROLLER,1336154020,50717,Acute Care Hospitals,7601 E IMPERIAL HWY,DOWNEY,Los Angeles,CA,90242,2,Medicare/Medicaid,3686626,2016,2018,MU,1382485,-118.156423,33.926359,325
325,REEDLEY COMMUNITY HOSPITAL,1336167550,50192,Acute Care Hospitals,372 W CYPRESS AVE,REEDLEY,Fresno,CA,93654,3,Medicare/Medicaid,2520247,2015,2018,MU,280027,-119.451265,36.607796,326
326,PROVIDENCE HEALTH SYSTEM-SOUTHERN CALIFORNIA,1336173269,50235,Acute Care Hospitals,501 S BUENA VISTA,BURBANK,Los Angeles,CA,91505,4,Medicare/Medicaid,2326084,2014,2015,MU,232608,-118.326997,34.155912,327


In [97]:

data.to_sql(MYTABLE, conn, index=False)

ValueError: Table 'kodurus_data' already exists.

In [98]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (328,19))
assert(list(dxyz.columns) == ['Provider_Name', 'NPI', 'CCN',
       'Medicaid_EP_Hospital_Type', 'Street_Address', 'City', 'County',
       'State', 'Zip_Code', 'Payment_Year_Number', 'Program_Type',
       'Total_payments', 'Last_Program_Year', 'Last_Payment_Year',
       'Last_Payment_Criteria', 'Most_Recent_Disbursement_Amount', 'LONGITUDE',
       'LATITUDE', 'FID'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Zip_Code` field and `population`'s `zipcode` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [99]:
### SOLUTION GOES HERE
dataset = pd.read_sql("SELECT * FROM " + MYTABLE + " JOIN population ON " + MYTABLE + ".Zip_Code = population.zipcode", conn)

zip = dataset.sort_values(by='population', ascending=False).reset_index()[['population', 'zipcode']]
zip_code = zip.zipcode[0]

answer = list(dataset[dataset['zipcode'] == zip_code]['NPI'].astype('str'))

display(answer)

['1194016923']

In [100]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [101]:
### SOLUTION GOES HERE
payment_totals = data.groupby(['Zip_Code','Medicaid_EP_Hospital_Type'])['Total_payments']
total_sums = payment_totals.sum().reset_index()

#display(total_sums)

population = pd.read_sql_query('SELECT * FROM ' + 'population', conn)

population['zipcode'] = pd.to_numeric(population['zipcode'])

frame = total_sums.merge(population, right_on='zipcode', left_on='Zip_Code', how='inner')

#display(frame)

In [102]:
frame['Avg_Pay_per_Capita'] = frame.loc[:,'Total_payments'] / frame.loc[:,'population']

answer = frame.groupby('Medicaid_EP_Hospital_Type')['Avg_Pay_per_Capita'].mean().reset_index()

display(answer)

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,17974.362790
1,Children's Hospitals,111.559021


In [103]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 17974.363)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 111.559)
